In [1]:
from artistplaylist import ArtistPlaylist
import event_scraper
import pprint

events = event_scraper.get_events(1, 10)
print("Found {0} events".format(len(events)))

Found 10 events


In [2]:
for ev in events:
    print(ev['artist'])

Phoebe Bridgers
Fury Hc
Forced Order
Lock
Glory
Unexpressed
Fury
Orphan
The Broken Crowns Tour
Charlotte DJ Set


In [3]:
ap = ArtistPlaylist()

song_ids = []
for ev in events:
    songs = ap.songs_from_artist(ev['artist'])
    result = None
    sid = None
    if songs:
        sid = songs[0]['storeId']
        song_ids.append(sid)
        result = songs[0]['genre']
    print(' -- '.join([ev['artist'], str(sid) , str(result)]))

INFO:gmusicapi.Mobileclient1:!-- begin debug log --!
INFO:gmusicapi.Mobileclient1:version: 10.1.2
INFO:gmusicapi.Mobileclient1:logging to: C:\Users\lanca_000\AppData\Local\Simon Weber\gmusicapi\Logs\gmusicapi.log
INFO:gmusicapi.Mobileclient1:initialized
INFO:gmusicapi.Mobileclient1:logged out
INFO:gmusicapi.Mobileclient1:authenticated
INFO:gmusicapi.Mobileclient1:subscribed


Phoebe Bridgers -- Twzqti3pd6qrujy4yte7ya5hwom -- Alternative/Indie
Fury Hc -- None -- None
Forced Order -- Tvkijtkjsrm7c4pqdpkiif34mlm -- Alternative/Indie
Lock -- Tzhqjnsfa6arrwvsjoialp7nquy -- Pop
Glory -- Tc2erkdo6wqs5ecdjdoajxoncom -- Soundtracks
Unexpressed -- Tj2jnnwwrxhaborhurvmflhqivi -- Jazz
Fury -- Tb4jgpiqhkuxcrh3ink5wkglslu -- Rock
Orphan -- Tro6ruk2vb6n4psgwzy72dtmitu -- Classic Rock
The Broken Crowns Tour -- Ta4wpolxpk4albf3wee2slobzha -- Reggae
Charlotte DJ Set -- None -- None


In [5]:
import ipywidgets as widgets
from IPython.display import display
button_pub = widgets.Button(description="Publish Playlist")
button_del = widgets.Button(description="Delete Playlist")
display(button_pub, button_del)

PUBLISH_PUBLICALLY = True

def pub_button_clicked(b):
    ap.create_playlist(song_ids, 'Austin Live Artists - V0.1', public=PUBLISH_PUBLICALLY)
    print("Published playlist")
    
def del_button_clicked(b):
    ap.delete_playlist()
    print("Deleted playlist")

button_pub.on_click(pub_button_clicked)
button_del.on_click(del_button_clicked)

Button(description='Publish Playlist', style=ButtonStyle())

Button(description='Delete Playlist', style=ButtonStyle())

Published playlist
Deleted playlist
